In [ ]:
#import all the necessary libraries: pandsas, numpy, matplotlib, votekit (If not, downolad it via pip install <library>)




import pandas as pd
import numpy as np
import os
from votekit.elections import Borda, Plurality, STV
from votekit.cvr_loaders import load_scottish
from fairness_metric import sigma_IIA, sigma_IIA_STV,sigma_UM_STV, sigma_UM, sigma_IIA_winner, sigma_UM_winner_set
from Voting_rules import Ranked_Borda, Ranked_Plurality, Ranked_2_Approval,Ranked_3_Approval, Optimal_sigma_UM_voting_rule_ranking,Ranked_STV
from votekit.cvr_loaders import load_scottish  # your load function for Scottish profiles
import re
from tqdm import tqdm


#This script computes the fairness metrics for the Scottish election data using voting rules: Plurality, Borda, 2-Approval, and 3-Approval. Note that for the STV there is a separate script.

np.random.seed(2025)  # For reproducibility as ties are broken randomly (though ties are rare in the scottish data)

# make sure to install the required libraries if you haven't already

#  Define base path 
base_path = '/Users/ss2776/Downloads/scot-elex-main' #scottish data path

#  Define all voting rules and labels 
voting_rules = {
    "Plurality-Ranking": Ranked_Plurality,
    "Borda-Ranking": Ranked_Borda,
    "2-Approval-Ranking": Ranked_2_Approval,
    "3-Approval-Ranking": Ranked_3_Approval,
    "STV-Ranking" : Ranked_STV,
    "Greedy_Sort- Ranking":Optimal_sigma_UM_voting_rule_ranking,
    "Plurality-Winner": Plurality,
    "Borda-Winner" : Borda,
    "STV-Winner" : STV
}

# Helper: parse council, year, ward from filename 
def split_file_name(file_name):
    parts = file_name.replace(".csv", "").split("_")
    council_words = []

    for i, part in enumerate(parts):
        if re.match(r"\d{4}", part):  # year pattern
            year = part
            council = "_".join(council_words)
            ward = "_".join(parts[i+1:])
            return council, int(year), ward
        else:
            council_words.append(part)

    # fallback
    return "_".join(parts[:-2]), int(parts[-2]), parts[-1]

# Main processing 
records = []

for num_cands in range(3, 15):
    folder_path = os.path.join(base_path, f"{num_cands}_cands")
    if not os.path.exists(folder_path):
        print(f"Skipping missing folder: {folder_path}")
        continue

    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    for file in tqdm(files, desc=f"{num_cands} candidates", leave=False):
        try:
            full_path = os.path.join(folder_path, file)
            profile, seats, cand_list, cand_to_party, _ = load_scottish(full_path)
            council, year, ward = split_file_name(file)

            result = {
                "Council": council,
                "Year": year,
                "Ward": ward,
                "Num_Candidates": num_cands,
                "Num_seats": seats,
            }

            # Compute and store metrics for each rule
            for rule_name, rule_func in voting_rules.items():
                if rule_name == "Plurality-Ranking" or rule_name == "Borda-Ranking" or rule_name == "2-Approval-Ranking" or rule_name == "3-Approval-Ranking" or rule_name == "Greedy_Sort- Ranking":

                    try:
                         result[f"Sigma_IIA_{rule_name}"] = round(float(sigma_IIA(profile, rule_func)),4)
                    except Exception as e:
                          
                          print(f"IIA error in {file} ({rule_name}): {e}")
                          result[f"Sigma_IIA_{rule_name}"] = None

                    try:
                        result[f"Sigma_UM_{rule_name}"] = round(float(sigma_UM(profile, rule_func)),4)
                    except Exception as e:
                        print(f"UF error in {file} ({rule_name}): {e}")
                        result[f"Sigma_UM_{rule_name}"] = None
                elif rule_name == "STV-Ranking":
                    try:
                         result[f"Sigma_IIA_{rule_name}"] = round(float(sigma_IIA_STV(profile, seats, rule_func)),4)
                    except Exception as e:
                          
                          print(f"IIA error in {file} ({rule_name}): {e}")
                          result[f"Sigma_IIA_{rule_name}"] = None

                    try:
                        result[f"Sigma_UM_{rule_name}"] = round(float(sigma_UM_STV(profile, seats,rule_func)),4)
                    except Exception as e:
                        print(f"UF error in {file} ({rule_name}): {e}")
                        result[f"Sigma_UM_{rule_name}"] = None
                elif rule_name == "Plurality-Winner" or rule_name == "Borda-Winner" or rule_name == "STV-Winner":
                    try:
                         result[f"Sigma_IIA_{rule_name}"] = round(float(sigma_IIA_winner(profile, rule_func, seats)),4)
                    except Exception as e:
                          
                          print(f"IIA error in {file} ({rule_name}): {e}")
                          result[f"Sigma_IIA_{rule_name}"] = None

                    try:
                        result[f"Sigma_UM_{rule_name}"] = round(float(sigma_UM_winner_set(profile, rule_func,seats)),4)
                    except Exception as e:
                        print(f"UF error in {file} ({rule_name}): {e}")
                        result[f"Sigma_UM_{rule_name}"] = None




            result["File"] = file.replace(".csv", "")
            records.append(result)

        except Exception as e:
            print(f"Error processing file {file}: {e}")

#  Create and save final DataFrame 
df = pd.DataFrame(records)

# reorder columns for readability
ordered_cols = [
    "Council", "Year", "Ward", "Num_Candidates", "Num_seats",
    "Sigma_IIA_Plurality", "Sigma_IIA_Borda", "Sigma_IIA_2-Approval", "Sigma_IIA_3-Approval",
    "Sigma_UM_Plurality", "Sigma_UM_Borda", "Sigma_UM_2-Approval", "Sigma_UM_3-Approval",
    "File"
]
#df_new = df[ordered_cols]

# Save output 
output_path = "/Users/ss2776/Downloads/all_scottish_complete_results.csv"  # specify your output path
df.to_csv(output_path, index=False)
print(f"\n Results saved to: {output_path}")

5 candidates:   6%|▋         | 7/110 [00:19<05:00,  2.91s/it]

In [1]:
from votekit.cvr_loaders import load_scottish
profile, seats, cand_list, cand_to_party, _ = load_scottish("scot-elex-main/6_cands/aberdeen_2022_ward7.csv")

In [4]:
from fairness_metric import sigma_IIA_STV
from Voting_rules import Ranked_STV

In [3]:
sigma_IIA_STV(profile, seats, Ranked_STV)

0.9